# 10 - Analyzing the Meaning of Sentences

Resource: https://www.nltk.org/book/ch10.html

In [81]:
import nltk

In [82]:
nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


In [83]:
from nltk import load_parser
cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

SELECT City FROM city_table WHERE Country="china"


In [84]:
from nltk.sem import chat80
rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows: print(r[0], end=" ")

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

In [85]:
nltk.boolean_ops()

negation       	-
conjunction    	&
disjunction    	|
implication    	->
equivalence    	<->


#### Boolen Operators in Propositional Logic
|Boolean Operator|Truth Conditions|
|:--------------:|:--------------:|
|negation (it is not the case that ...)|-φ is true in s	iff	φ is false in s|
|conjunction (and)|(φ & ψ) is true in s	iff	φ is true in s and ψ is true in s|
|disjunction (or)|(φ \| ψ) is true in s	iff	φ is true in s or ψ is true in s|
|implication (if ..., then ...)|(φ -> ψ) is true in s	iff	φ is false in s or ψ is true in s|
|equivalence (if and only if)|(φ <-> ψ) is true in s	iff	φ and ψ are both true in s or both false in s|


In [86]:
read_expr = nltk.sem.Expression.fromstring
read_expr('-(P & Q)')

<NegatedExpression -(P & Q)>

In [87]:
read_expr('P & Q')

<AndExpression (P & Q)>

In [88]:
read_expr('P | (R -> Q)')

<OrExpression (P | (R -> Q))>

In [89]:
read_expr('P <-> --P')

<IffExpression (P <-> --P)>

In [90]:
val = nltk.Valuation([('P', True), ('Q', True), ('R', False)])
val['R']

False

In [91]:
dom = set()
g = nltk.Assignment(dom)
m = nltk.Model(dom, val)

In [92]:
print(m.evaluate('(P & Q)', g))
print(m.evaluate('-(P & Q)', g))
print(m.evaluate('(P & R)', g))
print(m.evaluate('(P | Q)', g))

True
False
False
True


In [93]:
read_expr = nltk.sem.Expression.fromstring
expr = read_expr('walk(angus)', type_check=True)
expr

<ApplicationExpression walk(angus)>

In [94]:
expr.argument.type

e

In [95]:
expr.function

<ConstantExpression walk>

In [96]:
expr.function.type

<e,?>

In [97]:
sig = {'walk':'<e, t>'}
expr = read_expr('walk(angus)', signature=sig)
expr.function.type

e

In [98]:
read_expr = nltk.sem.Expression.fromstring
read_expr('dog(cyril)').free()

set()

In [99]:
read_expr('dog(x)').free()

{Variable('x')}

In [100]:
read_expr('own(angus, cyril)').free()

set()

In [101]:
read_expr('exists x.dog(x)').free()

set()

In [102]:
read_expr('((some x.walk(x)) -> sing(x))').free()

{Variable('x')}

In [103]:
read_expr('exists x.own(y, x)').free()

{Variable('y')}

#### First Order Logic Operation
|Operation|Description|
|:-------:|:---------:|
|=|equality|
|!=|inequality|
|exists|existential quantifier|
|all|universal quantifier|
|e.free()|show free variables of e|
|e.simplify()|carry out β-reduction on e|

In [104]:
dom = {'b', 'o', 'c'}

In [105]:
v = """
bertie => b
olive => o
cyril => c
boy => {b}
girl => {o}
dog => {c}
walk => {o, c}
see => {(b, o), (c, b), (o, c)}
"""
val = nltk.Valuation.fromstring(v)
print(val)

{'bertie': 'b',
 'boy': {('b',)},
 'cyril': 'c',
 'dog': {('c',)},
 'girl': {('o',)},
 'olive': 'o',
 'see': {('b', 'o'), ('c', 'b'), ('o', 'c')},
 'walk': {('c',), ('o',)}}


In [106]:
('o', 'c') in val['see']

True

In [107]:
('b',) in val['boy']

True

In [108]:
g = nltk.Assignment(dom, [('x', 'o'), ('y', 'c')])
g

{'x': 'o', 'y': 'c'}

In [109]:
print(g)

g[c/y][o/x]


In [110]:
m = nltk.Model(dom, val)
m.evaluate('see(olive, y)', g)

True

In [111]:
g['y']

'c'

In [112]:
m.evaluate('see(y, x)', g)

False

In [113]:
g.purge()
g

{}

In [114]:
m.evaluate('see(olive, y)', g)

'Undefined'

In [115]:
m.evaluate('see(bertie, olive) & boy(bertie) & -walk(bertie)', g)

True

In [116]:
m.evaluate('exists x.(girl(x) & walk(x))', g)

True

In [117]:
m.evaluate('girl(x) & walk(x)', g.add('x', 'o'))

True

In [118]:
fmla1 = read_expr('girl(x) | boy(x)')
m.satisfiers(fmla1, 'x', g)

{'b', 'o'}

In [119]:
fmla2 = read_expr('girl(x) -> walk(x)')
m.satisfiers(fmla2, 'x', g)

{'b', 'c', 'o'}

In [120]:
fmla3 = read_expr('walk(x) -> girl(x)')
m.satisfiers(fmla3, 'x', g)

{'b', 'o'}

In [121]:
m.evaluate('all x.(girl(x) -> walk(x))', g)

True

In [122]:
v2 = """
bruce => b
elspeth => e
julia => j
matthew => m
person => {b, e, j, m}
admire => {(j, b), (b, b), (m, e), (e, m)}
"""
val2 = nltk.Valuation.fromstring(v2)

In [123]:
dom2 = val2.domain
m2 = nltk.Model(dom2, val2)
g2 = nltk.Assignment(dom2)
fmla4 = read_expr('(person(x) -> exists y.(person(y) & admire(x, y)))')
m2.satisfiers(fmla4, 'x', g2)

{'b', 'e', 'j', 'm'}

In [124]:
fmla5 = read_expr('(person(y) & all x.(person(x) -> admire(x, y)))')
m2.satisfiers(fmla5, 'y', g2)
set()

set()

In [125]:
fmla6 = read_expr('(person(y) & all x.((x = bruce | x = julia) -> admire(x, y)))')
m2.satisfiers(fmla6, 'y', g2)

{'b'}

In [126]:
read_dexpr = nltk.sem.DrtExpression.fromstring
drs1 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
print(drs1)
drs1.draw()

([x,y],[angus(x), dog(y), own(x,y)])


In [127]:
print(drs1.fol())

exists x y.(angus(x) & dog(y) & own(x,y))


In [128]:
drs2 = read_dexpr('([x], [walk(x)]) + ([y], [run(y)])')
print(drs2)

(([x],[walk(x)]) + ([y],[run(y)]))


In [129]:
print(drs2.simplify())

([x,y],[walk(x), run(y)])


In [130]:
drs3 = read_dexpr('([], [(([x], [dog(x)]) -> ([y],[ankle(y), bite(x, y)]))])')
print(drs3.fol())

all x.(dog(x) -> exists y.(ankle(y) & bite(x,y)))


In [131]:
drs4 = read_dexpr('([x, y], [angus(x), dog(y), own(x, y)])')
drs5 = read_dexpr('([u, z], [PRO(u), irene(z), bite(u, z)])')
drs6 = drs4 + drs5
print(drs6.simplify())

([u,x,y,z],[angus(x), dog(y), own(x,y), PRO(u), irene(z), bite(u,z)])


In [132]:
print(drs6.simplify().resolve_anaphora())

([u,x,y,z],[angus(x), dog(y), own(x,y), (u = [x,y,z]), irene(z), bite(u,z)])


In [133]:
parser = nltk.load_parser('grammars/book_grammars/drt.fcfg', logic_parser=nltk.sem.drt.DrtParser())
trees = list(parser.parse('Angus owns a dog'.split()))
print(trees[0].label()['SEM'].simplify())

([x,z4],[Angus(x), dog(z4), own(x,z4)])
